In [ ]:
#lib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
doc =  '../Data/Clean/Binance_1h_BTCUSDT_2020-01-01_2025-07-07.csv'
name = '../Data/Clean/Binance_1h_BTCUSDT_2020-01-01_2025-07-07.csv'
csv = pd.read_csv(doc)
csv


In [ ]:
csv = csv.drop(columns = [cols for cols in csv.columns if cols.endswith('Unnamed: 0')])
csv

In [ ]:
import pandas as pd

# 讀取 CSV 檔案
t_df = pd.read_csv('tf.csv', parse_dates=['datetime'])
u_df = pd.read_csv('u.csv', parse_dates=['datetime'])

# 合併（根據 datetime 欄位，inner join 避免超過的被保留）
merged_df = pd.merge(t_df, u_df[['datetime', 'u']], on='datetime', how='inner')

# 儲存新的結果
merged_df.to_csv('merged.csv', index=False)


In [ ]:
#timestamp to datetime 
# csv['datetime'] = pd.to_datetime(csv['start_time'],unit = "ms") # coinglass

csv['datetime'] = pd.to_datetime(csv['Open time'] ) # binance
csv['datetime'] = csv['datetime'].dt.tz_localize(None)
if f'Unnamed: 0' in csv.columns:
   csv.drop(columns= 'Unnamed: 0',inplace= True)

csv


In [ ]:
csv = csv.drop(columns= [cols for cols in csv.columns if cols.startswith('Ignore')])


In [ ]:
csv

In [ ]:
min_date = csv['datetime'].min()
max_date = csv['datetime'].max()
full_range = pd.date_range(start=min_date, end=max_date, freq='1H') #1H , 1D 1min
csv.set_index('datetime')
csv.reindex(full_range)
csv.to_csv(name)

In [ ]:

full_range = pd.date_range(start=min_date, end=max_date, freq='1min') #1H , 1D
csv.set_index('Date')
csv.reindex(full_range)
csv.index.name = 'Date'
csv

In [ ]:
missing = csv.isnull().sum()
print(f'missing_places{missing}')



All_csv


In [ ]:
import os
import glob
import pandas as pd

folder_path = "../Data/close/"
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

for f in csv_files:
    # 正確讀取 CSV，並將 'Open time' 當作日期欄位
    csv = pd.read_csv(f, parse_dates=['Open time'])

    # 處理 datetime 欄位，去除時區
    csv['datetime'] = csv['Open time'].dt.tz_localize(None)

    # 移除多餘欄位
    if 'Unnamed: 0' in csv.columns:
        csv.drop(columns='Unnamed: 0', inplace=True)

    # 建立完整時間範圍
    min_date = csv['datetime'].min()
    max_date = csv['datetime'].max()

    if pd.isna(min_date) or pd.isna(max_date):
        print(f"[⚠️ 無法處理檔案] {f} -> datetime 欄位無效")
        continue

    full_range = pd.date_range(start=min_date, end=max_date, freq='1H')

    # 補齊時間索引
    csv.set_index('datetime', inplace=True)
    csv = csv[~csv.index.duplicated(keep='first')]
    csv = csv.reindex(full_range)
    csv.index.name = 'datetime'

    # 儲存
    name_only = os.path.basename(f)
    output_path = os.path.join("../Data/close/", name_only)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    csv.to_csv(output_path)